In [ ]:
pip install pandas influxdb-client

In [1]:
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd


# InfluxDB Connection Details
INFLUXDB_URL = "https://influxdb-battery-demo.apps.ocp4.rhlab.de/"
INFLUXDB_TOKEN = "admin_token"
INFLUXDB_ORG = "redhat"
INFLUXDB_BUCKET = "bms"

In [2]:
# Initialize Client
client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)
write_api = client.write_api(write_options=SYNCHRONOUS)

def retrieve_battery_data():
    query = f'''
    from(bucket: "{INFLUXDB_BUCKET}")
      |> range(start: -1h)
      |> filter(fn: (r) => r["_measurement"] == "battery_data")
    '''
    query_api = client.query_api()
    tables = query_api.query(query, org=INFLUXDB_ORG)

    # Process Results
    data = []
    for table in tables:
        for record in table.records:
            data.append(record.values)

    df = pd.DataFrame(data)
    print(df)  # Print the retrieved data
    return df

In [3]:
df = retrieve_battery_data()

# Drop non-numeric columns
#df = df.drop(columns=["result", "table", "_start", "_stop", "_measurement", "_time"])
df = df.drop(columns=["result", "table", "_measurement"])

# Save for ML
df.to_csv("./data/battery_raw.csv", index=False)
print("Saved raw battery data from InfluxDB.")

       result  table                           _start  \
0     _result      0 2025-02-12 13:52:08.151214+00:00   
1     _result      0 2025-02-12 13:52:08.151214+00:00   
2     _result      0 2025-02-12 13:52:08.151214+00:00   
3     _result      0 2025-02-12 13:52:08.151214+00:00   
4     _result      0 2025-02-12 13:52:08.151214+00:00   
...       ...    ...                              ...   
4043  _result      7 2025-02-12 13:52:08.151214+00:00   
4044  _result      7 2025-02-12 13:52:08.151214+00:00   
4045  _result      7 2025-02-12 13:52:08.151214+00:00   
4046  _result      7 2025-02-12 13:52:08.151214+00:00   
4047  _result      7 2025-02-12 13:52:08.151214+00:00   

                                _stop                            _time  \
0    2025-02-12 14:52:08.151214+00:00 2025-02-12 14:26:34.190000+00:00   
1    2025-02-12 14:52:08.151214+00:00 2025-02-12 14:26:37.025000+00:00   
2    2025-02-12 14:52:08.151214+00:00 2025-02-12 14:26:40.020000+00:00   
3    2025-02-12 14: